In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt
from IPython.display import display
##
import json
import re
import numpy           as np
import pandas          as pd
import statsmodels.api as sm
import ipywidgets      as widgets
## Shut up matplotlib
plt.rcParams.update({'figure.max_open_warning': 0})

Rather messy notebook for studying performance of different implementations of hashes

In [ ]:
def read_bench(fname):
    "Read benchmarks from CSV file produced by criterion"
    df = pd.read_csv(fname)
    df['bytes']   = df['Name'].apply(lambda s: int(s.split()[1]))
    df['Name']    = df['Name'].apply(lambda s: s.split()[0].replace('Hash/','').replace('/blob',''))
    df['MeanErr'] = (df['MeanUB'] - df['MeanLB']) / 2
    return { k:{'df':v} for k,v in df.groupby('Name')}

def fit_bench(df):
    "Perform linear fit on benchmark data"
    return sm.OLS( df['Mean'], sm.add_constant(df['bytes'])).fit()

def read_and_fit(fname):
    return {
        k : d.update({'fit': fit_bench(d['df']),
                      'name': k
                     }) or d
        for k,d in read_bench(fname).items()    
    }

def hash_summary(meas) :
    "Generate summary of benchmark"
    return pd.DataFrame.from_records([
        [d['name'],
         1e9*d['fit'].params[0],
         1e9*d['fit'].params[1]
        ] for _,d in meas.items() ], 
        columns=['name','t0','tByte'])

In [ ]:
meas_pure       = read_and_fit('../pure.csv')
meas_cryptonite = read_and_fit('../cryptonite.csv')
sum_pure        = hash_summary(meas_pure)
sum_cryptonite  = hash_summary(meas_cryptonite)

In [ ]:
def plot_meas(meas) :
    df = meas['df']
    r  = meas['fit']
    plt.figure()
    plt.grid()
    plt.title("%s: %.2f + %0.2f·t ns" % (meas['name'], 1e9*r.params[0], 1e9*r.params[1]))
    plt.semilogx()
    plt.semilogy()
    plt.plot(df['bytes'], df['Mean'], '+')
    # Model
    xs = np.logspace(0, np.log10(np.max(df['bytes'])))
    plt.plot(xs, r.params[0] + r.params[1]*xs)    

In [ ]:
display(sum_pure)
display(sum_cryptonite)

In [ ]:
for k,d in meas_pure.items():
    plot_meas(d)

In [ ]:
for k,d in meas_cryptonite.items():
    plot_meas(d)

In [ ]:
sum_pure['tByte'] / sum_cryptonite['tByte']